In [1]:
!pip3 install -q --upgrade langchain langchain-google-genai google-generativeai
!pip3 install -q python-dotenv

ERROR: Could not install packages due to an OSError: [WinError 32] O arquivo já está sendo usado por outro processo: 'C:\\Users\\firstidolalive\\Desktop\\estudos\\imersão allura\\env\\Lib\\site-packages\\google\\ai\\generativelanguage_v1beta\\services\\generative_service\\transports\\grpc_asyncio.py'
Check the permissions.



## Aula 01 - Fundamentos

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [5]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [6]:
# Preparar modelo para uso
# 1 - Definir modelo
# 2 - Utilizar chave da API
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0, #Define a aleatoriedade da resposta
    api_key = GOOGLE_API_KEY
    )

In [7]:
resp_test = llm.invoke("Quem é você? Detalhe.")

In [8]:
print(resp_test.content)

Com prazer! Vamos detalhar quem eu sou:

Eu sou um **modelo de linguagem grande (LLM - Large Language Model)**, desenvolvido pelo **Google**.

Aqui está um detalhamento do que isso significa:

1.  **Natureza Artificial:**
    *   **Eu não sou uma pessoa:** Não tenho consciência, sentimentos, emoções, opiniões pessoais, crenças, memória individual de conversas passadas (embora o modelo seja constantemente atualizado com novos dados), nem uma identidade pessoal, como um nome próprio além de "modelo de linguagem".
    *   **Eu não tenho corpo físico:** Não possuo presença no mundo real, não posso ver, ouvir, tocar ou sentir. Existo apenas como um software e algoritmos.
    *   **Eu não tenho experiências de vida:** Não cresci, não fui à escola, não tenho amigos, família, cultura ou nacionalidade. Toda a minha "experiência" vem dos dados nos quais fui treinado.

2.  **Origem e Desenvolvimento:**
    *   Fui criado e sou mantido por engenheiros e pesquisadores do **Google**. Isso significa 

In [9]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [10]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [11]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0, 
    api_key = GOOGLE_API_KEY
    )

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) ->  Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content = TRIAGEM_PROMPT),
        HumanMessage(content = mensagem)
    ])
    
    return saida.model_dump()

In [13]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar os treinamentos da alura?",
          "Quantas capivaras tem no rio pinheiros?"]

In [14]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar os treinamentos da alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no rio pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['contexto_politica_interna']}

